You can think of the discriminator (D) output as a probability density function, you give it an image, if it is real, the probability is close to 1 and if not close to 0. The shape of pdf(x) is very nonlinear of course. The job of the generator is to map input noise to an output x - which can be an image, where pdf(x) is high. Think of x as a random variable with a pdf like normal-pdf or weibull-pdf. In parametrical statistics, we would first a proabilistic assumption about x's like they come from a normal pdf. Then we will take mean and varianve of sample of x's and that allow us to estimate the pdf of x. here we are trying to estimate any general pdf and then generate samples from this distribution.


This process is similar to importance/gibbs/MHasting sampling where we draw samples and then reject or accept them depending on a criteria. 

Equilibirum is a point in the game where neither players can do any better

# Generator Training Logic:
I think it is faily easy to understand how the discriminator loss works. The generator loss is tricker, because the course instructions keeps saying it is trained on the flipped labels. 

this is how i explain it.
first during training of generator we only update the parameters of the generator. We do not mess with discriminator parameters, as shown below:
```
        # Update the parameters in the optimizer.
        g_optimizer.step()
```
so we start with some random inputs, and then pass it to the discriminator for final score. 
so what happens to z2 ? it gets bigger and bigger inside Generator, but then when it gets to Discriminator, it shrinks down to eventually one score [0, 1] as shown below (look at the trpazoids)

<img src='assets/gan_pipeline.png' width=70% />

here is the code that sends z2 to generator and then to the discriminator. 

```
        # Generate fake images
        z2 = np.random.uniform(-1, 1, size=(batch_size, z_size))
        z2 = torch.from_numpy(z2).float()
        z2 = z2.cuda()
        generator_training_fake_images = G(z2)
        # Compute the discriminator losses on fake images 
        # using flipped labels!
        discriminator_score_fake_images = D(generator_training_fake_images)
```
We would like to train the Generator so the final score from discriminator is 1. That is, if the score is far from 1 currently and closer to 0, we want to train Generator parameters during generator training (not the Discriminator parameters), so that the discriminator final score becomes close to 1. So we adjust the initial layers that belong to Generator to accomplish this since we cannot touch the Discriminator at this point. This is  exactly the opposite of transfer learning, for example in Resnet, in which we adjust the final layers of a deep network, as opposed to initial layers here, to achieve higher classification performance. In transger learning we do not touch the initial layers, and can only train the final layers. 

notice if you include the discriminator in the training of generator with labels of 1, you would essentially train the discriminator (along with generator) to say any garbage that generator produces is real. So it is critical you do not include discriminator parameters during generator training. 

